# Data Manipulation with Polars Using The Fuel Economy Dataset

### Loading Libraries

In [1]:
# ZipFiles
import zipfile

# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd
import polars as pl
import polars.selectors as cs

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
pl.__version__

'1.14.0'

### Getting Data

In [3]:
# Path
path = '/Users/isisromero/desktop/polars/datasets/vehicles.csv'

In [4]:
raw = pl.read_csv(path, null_values=['NA'])

#### Placing `tweak_auto` Function

In [5]:
def tweak_auto(df):
    cols = ['year', 'make', 'model', 'displ', 'cylinders', 'trany', 
            'drive', 'VClass', 'fuelType', 'barrels08', 'city08', 'highway08', 'createdOn']
    return (df
            .select(pl.col(cols))
            .with_columns(pl.col('year').cast(pl.Int16),
                          pl.col(['cylinders', 'highway08', 'city08']).cast(pl.UInt8),
                          pl.col(['displ', 'barrels08']).cast(pl.Float32),
                          pl.col(['make', 'model', 'VClass', 'drive', 'fuelType']).cast(pl.Categorical),
                          pl.col('createdOn').str.to_datetime('%a %b %d %H:%M:%S %Z %Y'),
                          is_automatic=pl.col('trany')                    
                          .str.contains('Automatic')
                          .fill_null('Automatic'),
                          num_gears=pl.col('trany')
                          .str.extract(r'(\d+)')
                          .cast(pl.UInt8)
                          .fill_null(6))
           )

In [6]:
autos = tweak_auto(raw)

In [7]:
print(autos)

shape: (48_231, 15)
┌──────┬────────────┬──────────────┬───────┬───┬───────────┬─────────────┬─────────────┬───────────┐
│ year ┆ make       ┆ model        ┆ displ ┆ … ┆ highway08 ┆ createdOn   ┆ is_automati ┆ num_gears │
│ ---  ┆ ---        ┆ ---          ┆ ---   ┆   ┆ ---       ┆ ---         ┆ c           ┆ ---       │
│ i16  ┆ cat        ┆ cat          ┆ f32   ┆   ┆ u8        ┆ datetime[μs ┆ ---         ┆ u8        │
│      ┆            ┆              ┆       ┆   ┆           ┆ ]           ┆ str         ┆           │
╞══════╪════════════╪══════════════╪═══════╪═══╪═══════════╪═════════════╪═════════════╪═══════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider       ┆ 2.0   ┆ … ┆ 25        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆            ┆ Veloce 2000  ┆       ┆   ┆           ┆ 00:00:00    ┆             ┆           │
│ 1985 ┆ Ferrari    ┆ Testarossa   ┆ 4.9   ┆ … ┆ 14        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆            ┆              ┆       ┆   ┆           ┆ 00:00:00  

### Adding Columns

In [10]:
print(autos
      .select(pl.col(['highway08', 'city08']),
              mpg_ratio=(pl.col('highway08') / pl.col('city08'))
             )
     )

shape: (48_231, 3)
┌───────────┬────────┬───────────┐
│ highway08 ┆ city08 ┆ mpg_ratio │
│ ---       ┆ ---    ┆ ---       │
│ u8        ┆ u8     ┆ f64       │
╞═══════════╪════════╪═══════════╡
│ 25        ┆ 19     ┆ 1.315789  │
│ 14        ┆ 9      ┆ 1.555556  │
│ 33        ┆ 23     ┆ 1.434783  │
│ 12        ┆ 10     ┆ 1.2       │
│ 23        ┆ 17     ┆ 1.352941  │
│ …         ┆ …      ┆ …         │
│ 26        ┆ 19     ┆ 1.368421  │
│ 28        ┆ 20     ┆ 1.4       │
│ 24        ┆ 18     ┆ 1.333333  │
│ 24        ┆ 18     ┆ 1.333333  │
│ 21        ┆ 16     ┆ 1.3125    │
└───────────┴────────┴───────────┘


In [13]:
# Adding mpg_ratio column to Dataframe
print(autos
      .with_columns(mpg_ratio=pl.col('highway08') / pl.col('city08'))
     )

shape: (48_231, 16)
┌──────┬────────────┬──────────────┬───────┬───┬─────────────┬─────────────┬───────────┬───────────┐
│ year ┆ make       ┆ model        ┆ displ ┆ … ┆ createdOn   ┆ is_automati ┆ num_gears ┆ mpg_ratio │
│ ---  ┆ ---        ┆ ---          ┆ ---   ┆   ┆ ---         ┆ c           ┆ ---       ┆ ---       │
│ i16  ┆ cat        ┆ cat          ┆ f32   ┆   ┆ datetime[μs ┆ ---         ┆ u8        ┆ f64       │
│      ┆            ┆              ┆       ┆   ┆ ]           ┆ str         ┆           ┆           │
╞══════╪════════════╪══════════════╪═══════╪═══╪═════════════╪═════════════╪═══════════╪═══════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider       ┆ 2.0   ┆ … ┆ 2013-01-01  ┆ false       ┆ 5         ┆ 1.315789  │
│      ┆            ┆ Veloce 2000  ┆       ┆   ┆ 00:00:00    ┆             ┆           ┆           │
│ 1985 ┆ Ferrari    ┆ Testarossa   ┆ 4.9   ┆ … ┆ 2013-01-01  ┆ false       ┆ 5         ┆ 1.555556  │
│      ┆            ┆              ┆       ┆   ┆ 00:00:00    ┆         

### Simulating The Index

In [14]:
# Creating & Simulating Index
print(autos
      .with_row_index('index')
     )

shape: (48_231, 16)
┌───────┬──────┬────────────┬──────────────┬───┬───────────┬─────────────┬─────────────┬───────────┐
│ index ┆ year ┆ make       ┆ model        ┆ … ┆ highway08 ┆ createdOn   ┆ is_automati ┆ num_gears │
│ ---   ┆ ---  ┆ ---        ┆ ---          ┆   ┆ ---       ┆ ---         ┆ c           ┆ ---       │
│ u32   ┆ i16  ┆ cat        ┆ cat          ┆   ┆ u8        ┆ datetime[μs ┆ ---         ┆ u8        │
│       ┆      ┆            ┆              ┆   ┆           ┆ ]           ┆ str         ┆           │
╞═══════╪══════╪════════════╪══════════════╪═══╪═══════════╪═════════════╪═════════════╪═══════════╡
│ 0     ┆ 1985 ┆ Alfa Romeo ┆ Spider       ┆ … ┆ 25        ┆ 2013-01-01  ┆ false       ┆ 5         │
│       ┆      ┆            ┆ Veloce 2000  ┆   ┆           ┆ 00:00:00    ┆             ┆           │
│ 1     ┆ 1985 ┆ Ferrari    ┆ Testarossa   ┆ … ┆ 14        ┆ 2013-01-01  ┆ false       ┆ 5         │
│       ┆      ┆            ┆              ┆   ┆           ┆ 00:00:00  

### Removing Columns

In [15]:
print(autos
      .drop('createdOn')
      .columns
     )

['year', 'make', 'model', 'displ', 'cylinders', 'trany', 'drive', 'VClass', 'fuelType', 'barrels08', 'city08', 'highway08', 'is_automatic', 'num_gears']


In [16]:
final_cols = ['year', 'make', 'model', 'displ', 'cylinders', 'trany', 
              'drive', 'VClass', 'fuelType', 'barrels08', 'city08', 'highway08', 
              'is_automatic', 'num_gears']

In [17]:
print(autos
      .select(final_cols)
     )

shape: (48_231, 14)
┌──────┬────────────┬──────────────────┬───────┬───┬────────┬───────────┬──────────────┬───────────┐
│ year ┆ make       ┆ model            ┆ displ ┆ … ┆ city08 ┆ highway08 ┆ is_automatic ┆ num_gears │
│ ---  ┆ ---        ┆ ---              ┆ ---   ┆   ┆ ---    ┆ ---       ┆ ---          ┆ ---       │
│ i16  ┆ cat        ┆ cat              ┆ f32   ┆   ┆ u8     ┆ u8        ┆ str          ┆ u8        │
╞══════╪════════════╪══════════════════╪═══════╪═══╪════════╪═══════════╪══════════════╪═══════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider Veloce    ┆ 2.0   ┆ … ┆ 19     ┆ 25        ┆ false        ┆ 5         │
│      ┆            ┆ 2000             ┆       ┆   ┆        ┆           ┆              ┆           │
│ 1985 ┆ Ferrari    ┆ Testarossa       ┆ 4.9   ┆ … ┆ 9      ┆ 14        ┆ false        ┆ 5         │
│ 1985 ┆ Dodge      ┆ Charger          ┆ 2.2   ┆ … ┆ 23     ┆ 33        ┆ false        ┆ 5         │
│ 1985 ┆ Dodge      ┆ B150/B250 Wagon  ┆ 5.2   ┆ … ┆ 10     ┆ 12       

### Renaming Columns

In [20]:
print(autos
      .select(pl.all()
              .exclude(['createdOn', 'barrels08'])
              .name.suffix('_auto'))
             )

shape: (48_231, 13)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ year_auto ┆ make_auto ┆ model_aut ┆ displ_aut ┆ … ┆ city08_au ┆ highway08 ┆ is_automa ┆ num_gear │
│ ---       ┆ ---       ┆ o         ┆ o         ┆   ┆ to        ┆ _auto     ┆ tic_auto  ┆ s_auto   │
│ i16       ┆ cat       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│           ┆           ┆ cat       ┆ f32       ┆   ┆ u8        ┆ u8        ┆ str       ┆ u8       │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 1985      ┆ Alfa      ┆ Spider    ┆ 2.0       ┆ … ┆ 19        ┆ 25        ┆ false     ┆ 5        │
│           ┆ Romeo     ┆ Veloce    ┆           ┆   ┆           ┆           ┆           ┆          │
│           ┆           ┆ 2000      ┆           ┆   ┆           ┆           ┆           ┆          │
│ 1985      ┆ Ferrari   ┆ Testaross ┆ 4.9       ┆ … ┆ 9         ┆ 14   

In [21]:
final_cols = ['year', 'make', 'model', 'city_mpg', 'highway_mpg']

In [24]:
print(autos
      .with_columns(pl.col('city08').alias('city_mpg'),
                    highway_mpg=pl.col('highway08'))
      .select(final_cols)
     )

shape: (48_231, 5)
┌──────┬────────────┬─────────────────────┬──────────┬─────────────┐
│ year ┆ make       ┆ model               ┆ city_mpg ┆ highway_mpg │
│ ---  ┆ ---        ┆ ---                 ┆ ---      ┆ ---         │
│ i16  ┆ cat        ┆ cat                 ┆ u8       ┆ u8          │
╞══════╪════════════╪═════════════════════╪══════════╪═════════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider Veloce 2000  ┆ 19       ┆ 25          │
│ 1985 ┆ Ferrari    ┆ Testarossa          ┆ 9        ┆ 14          │
│ 1985 ┆ Dodge      ┆ Charger             ┆ 23       ┆ 33          │
│ 1985 ┆ Dodge      ┆ B150/B250 Wagon 2WD ┆ 10       ┆ 12          │
│ 1993 ┆ Subaru     ┆ Legacy AWD Turbo    ┆ 17       ┆ 23          │
│ …    ┆ …          ┆ …                   ┆ …        ┆ …           │
│ 1993 ┆ Subaru     ┆ Legacy              ┆ 19       ┆ 26          │
│ 1993 ┆ Subaru     ┆ Legacy              ┆ 20       ┆ 28          │
│ 1993 ┆ Subaru     ┆ Legacy AWD          ┆ 18       ┆ 24          │
│ 1993 ┆ Subaru

##### `SyntaxError` due to keyword argument

In [25]:
print(autos
...  .with_columns(highway_mpg=pl.col('highway08'),
...                pl.col('city08').alias('city_mpg'))
...  .select(final_cols)
...  )

SyntaxError: positional argument follows keyword argument (2241669681.py, line 3)

#### `.rename` as a Dictionary Mapping

In [26]:
final_cols = ['year', 'make', 'model', 'city_mpg', 'highway_mpg']

In [27]:
print(autos
      .rename({'city08':'city_mpg', 'highway08':'highway_mpg'})
      .select(final_cols)
     )

shape: (48_231, 5)
┌──────┬────────────┬─────────────────────┬──────────┬─────────────┐
│ year ┆ make       ┆ model               ┆ city_mpg ┆ highway_mpg │
│ ---  ┆ ---        ┆ ---                 ┆ ---      ┆ ---         │
│ i16  ┆ cat        ┆ cat                 ┆ u8       ┆ u8          │
╞══════╪════════════╪═════════════════════╪══════════╪═════════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider Veloce 2000  ┆ 19       ┆ 25          │
│ 1985 ┆ Ferrari    ┆ Testarossa          ┆ 9        ┆ 14          │
│ 1985 ┆ Dodge      ┆ Charger             ┆ 23       ┆ 33          │
│ 1985 ┆ Dodge      ┆ B150/B250 Wagon 2WD ┆ 10       ┆ 12          │
│ 1993 ┆ Subaru     ┆ Legacy AWD Turbo    ┆ 17       ┆ 23          │
│ …    ┆ …          ┆ …                   ┆ …        ┆ …           │
│ 1993 ┆ Subaru     ┆ Legacy              ┆ 19       ┆ 26          │
│ 1993 ┆ Subaru     ┆ Legacy              ┆ 20       ┆ 28          │
│ 1993 ┆ Subaru     ┆ Legacy AWD          ┆ 18       ┆ 24          │
│ 1993 ┆ Subaru

### Joining Data Frames

In [28]:
trucks = pl.DataFrame(
    {'make': ['Ford', 'Tesla', 'Chevy', 'Custom', 'Ford'],
     'model': ['F150', 'Cybertruck', 'Silverado', 'HotRod', 'F250'],
     'year': [2018, 2024, 2019, 1967, 2017],
     'city_mpg': [19, None, 17, 12, 18],
    })

In [30]:
print(trucks)

shape: (5, 4)
┌────────┬────────────┬──────┬──────────┐
│ make   ┆ model      ┆ year ┆ city_mpg │
│ ---    ┆ ---        ┆ ---  ┆ ---      │
│ str    ┆ str        ┆ i64  ┆ i64      │
╞════════╪════════════╪══════╪══════════╡
│ Ford   ┆ F150       ┆ 2018 ┆ 19       │
│ Tesla  ┆ Cybertruck ┆ 2024 ┆ null     │
│ Chevy  ┆ Silverado  ┆ 2019 ┆ 17       │
│ Custom ┆ HotRod     ┆ 1967 ┆ 12       │
│ Ford   ┆ F250       ┆ 2017 ┆ 18       │
└────────┴────────────┴──────┴──────────┘


In [31]:
manufacturer = pl.DataFrame(
    {'name': ['Ford', 'Tesla', 'Chevy', 'Toyota'],
     'country': ['USA', 'USA', 'USA', 'Japan'],
     'founded': [1903, 2003, 1911, 1937],
     'employees': [199_000, 48_000, 225_000, 370_000],
     'vehicles': [80, 3, 45, 30],
    })

In [32]:
print(manufacturer)

shape: (4, 5)
┌────────┬─────────┬─────────┬───────────┬──────────┐
│ name   ┆ country ┆ founded ┆ employees ┆ vehicles │
│ ---    ┆ ---     ┆ ---     ┆ ---       ┆ ---      │
│ str    ┆ str     ┆ i64     ┆ i64       ┆ i64      │
╞════════╪═════════╪═════════╪═══════════╪══════════╡
│ Ford   ┆ USA     ┆ 1903    ┆ 199000    ┆ 80       │
│ Tesla  ┆ USA     ┆ 2003    ┆ 48000     ┆ 3        │
│ Chevy  ┆ USA     ┆ 1911    ┆ 225000    ┆ 45       │
│ Toyota ┆ Japan   ┆ 1937    ┆ 370000    ┆ 30       │
└────────┴─────────┴─────────┴───────────┴──────────┘


In [33]:
print(manufacturer.join(trucks, how='left'))

ValueError: must specify `on` OR `left_on` and `right_on`

In [34]:
print(manufacturer.join(trucks, how='left', left_on='name', right_on='make'))

shape: (5, 8)
┌────────┬─────────┬─────────┬───────────┬──────────┬────────────┬──────┬──────────┐
│ name   ┆ country ┆ founded ┆ employees ┆ vehicles ┆ model      ┆ year ┆ city_mpg │
│ ---    ┆ ---     ┆ ---     ┆ ---       ┆ ---      ┆ ---        ┆ ---  ┆ ---      │
│ str    ┆ str     ┆ i64     ┆ i64       ┆ i64      ┆ str        ┆ i64  ┆ i64      │
╞════════╪═════════╪═════════╪═══════════╪══════════╪════════════╪══════╪══════════╡
│ Ford   ┆ USA     ┆ 1903    ┆ 199000    ┆ 80       ┆ F150       ┆ 2018 ┆ 19       │
│ Ford   ┆ USA     ┆ 1903    ┆ 199000    ┆ 80       ┆ F250       ┆ 2017 ┆ 18       │
│ Tesla  ┆ USA     ┆ 2003    ┆ 48000     ┆ 3        ┆ Cybertruck ┆ 2024 ┆ null     │
│ Chevy  ┆ USA     ┆ 1911    ┆ 225000    ┆ 45       ┆ Silverado  ┆ 2019 ┆ 17       │
│ Toyota ┆ Japan   ┆ 1937    ┆ 370000    ┆ 30       ┆ null       ┆ null ┆ null     │
└────────┴─────────┴─────────┴───────────┴──────────┴────────────┴──────┴──────────┘
